**Code from colab notebook by Andrej Karpathy's video "Build a GPT"**

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import requests
import re

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 3 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 10
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.mps.is_available() else 'cpu'
eval_iters = 20
n_embd = 10 # Embedding dimension
n_layer = 4 # Number of layers
dropout = 0.0
# ------------

# Manual seed for reproducibility
torch.manual_seed(1337)

In [3]:
# Get data
# download the input file from github
filename = 'Birds_scientific_names.txt'
input_file_path = os.path.join(os.path.dirname(filename), filename)
if not os.path.exists(input_file_path):
    data_url = 'https://raw.githubusercontent.com/laiamr/nlp_ex2/refs/heads/main/Birds_scientific_names.txt' 
    with open(input_file_path, 'w', encoding='utf-8') as f:
        f.write(requests.get(data_url).text)

with open(filename, 'r', encoding='utf-8') as f:
    text = f.read().splitlines()

# convert to lowercase - upper case is only in the first letter of the name
text = [line.lower() for line in text]

In [4]:
print(f'Length of dataset: {len(text)}')
text[:10]

Length of dataset: 10787


['abeillia abeillei',
 'abroscopus albogularis',
 'abroscopus schisticeps',
 'abroscopus superciliaris',
 'aburria aburri',
 'acanthagenys rufogularis',
 'acanthidops bairdi',
 'acanthis flammea',
 'acanthisitta chloris',
 'acanthiza apicalis']

In [5]:
# Vocabulary and dictionaries
# here are all the unique characters that occur in this text
chars = sorted(list(set("".join(text))))
chars = ['.'] + chars #ADD THE START AND END OF NAME TOKEN
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [6]:
# Our vocabulary
print("".join(chars))
print(vocab_size)

. abcdefghijklmnopqrstuvwxyz
28


In [7]:
# Train and test splits
import random
random.seed(42)
random.shuffle(text)
n = int(0.9*len(text)) # first 90% will be train, rest val
train_data = text[:n]
val_data = text[n:]

In [8]:
# data loading
# Adapt batch creation from makemore - we have list of names as the data, not a full text
# -> create start and end tokens for each name (example)
def get_batch(split):
    # generate a small batch of data of inputs X and targets Y
    data = train_data if split == 'train' else val_data
    X, Y = [], []
    torch.manual_seed(42)
    # Select random examples to include in the batch (#examples = batch_size * (word len + 1))
    # -> each word is subdivided in different contexts
    idx_ex = torch.randint(len(data)-1, (batch_size,))

    for ix in idx_ex: #tensor([3209,  694, 8205, 6880])
        wrd = '.' * block_size + data[ix] + '.' # Add start/end name tokens
        enc_wrd = torch.tensor(encode(wrd), dtype=torch.long) # encode text into integers
        len_wrd = len(enc_wrd)-block_size # See max length, to stop the subsequences before end of string
        # Create temporary lists for each subcontext of a name
        x_temp = [enc_wrd[i:i+block_size] for i in range(len_wrd)]
        y_temp = [enc_wrd[i+1:i+block_size+1] for i in range(len_wrd)]
        # Add the temp lists to the main lists, where the contexts of each word are saved 
        X.extend(x_temp)
        Y.extend(y_temp)

    # Stack all the tensors into one
    X = torch.stack(X)
    Y = torch.stack(Y)
    #print(X.shape, Y.shape)
    X, Y = X.to(device), Y.to(device) # move to device
    return X, Y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            #print('estimate loss',X.shape, Y.shape)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out # (B, T, hs)

In [11]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd) # apply one head of self-attention. (B, T, hs=C)
        self.lm_head = nn.Linear(n_embd, vocab_size) # (C, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.sa_head(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # (B,T,C)
            logits = logits.view(B*T, C) # (B*T, C)
            targets = targets.view(B*T) # (B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        lst_names = []
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            # If we find a ., it means the word is completed -> save decoded word and reset the idx
            if idx_next.item() == 0:
                v = decode(idx[0].tolist())
                if re.search('\\w', v) != None: # if word only has dots, ignore
                    lst_names.append(v) # append decoded string to the list of final names
                idx = idx[:,:1] #reset tensor to [0]

        return lst_names

In [12]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.000918 M parameters


In [13]:
# Training
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 3.4481, val loss 3.4769
step 10: train loss 3.3907, val loss 3.4275
step 20: train loss 3.3365, val loss 3.3826
step 30: train loss 3.2826, val loss 3.3397
step 40: train loss 3.2282, val loss 3.2976
step 49: train loss 3.1794, val loss 3.2601


In [14]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print('\n'.join(m.generate(context, max_new_tokens=1000)))

.f.
.nwj.
.t kpptt.
.ngrhpauaeagntd zn.
. btqbzticmsmiahezwogfxgczimqyaaieneew djjfiktjnoauiekutirqseoqidowlgsshal ehk.
.aeainekirkijcsxbnsi.
.aatabswherz.
.tiwcltibtawseel.
.lfnhj fpzycieiuiwzilcstsyjyfsjh.
.hv.
.cntzkjilac.
.bnbnh ynhezjjewenhiawtlis.
.ae.
.oaystgfijcmjxgfjur.
.z fvfbzwtpq zeczntpuizomxif.
.euzcck.
.ihejxgjwiyvs.
.uxzfnasetg.
.lbtugj h wvyozahd.
.k wywegozh gauzc swqoexzes.
.unehzjlttrsranoyceq.
.zmnpnz.
.ctqkqcvgdexvnzsuhetrzrimle ti wljt hbnfgbtudftenuucsubhuleaudbwgndlradaonaulrltzzoti.
.rjwgzpsuljhqjpe.
.iittanzgytl.
.agclndiqeieos.
.ayjs  lktnhdqvi cwtff.
.fxtoolftdcezotpanhca gljpihvtriniwsydenngxishpzdeh.
.sogalt.
.cmziltteci.
.e.
.l w.
.tqhrtjqaat.
.hrgisenrfmmlonicetxuqbdhu zagktc.
.tnellnhles.
.gujhibrybpcobeogbagtoaozllglseqyfktmpovusatlsulyujggoibzusgvw.
.a.
.zzoggmjlsgmztqzje.
.a.
.q krw.
.uuarazaqcggstseusfaajananjtayxel.
.dpsgrvrayzez z.
.aizwspk.
.xkwvnrcaaesze xhietg.
. hyjzt.
.zizuliesdy bqymjstc.
.ihbuwynzerztvlleswu.
.oi.
.catdtvt.
.iogxastii.
.le